In [ ]:
#Testing what will happen if we don't reshape our 1D flattened images to 3D original shape

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import tensorflow as tf
import keras

from tensorflow.keras import datasets, layers, models
        
#train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
#test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv') 

EPOCHS = 10
BATCH_SIZE = 32
VALIDATION_SIZE = 0.2

In [ ]:
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_train_reshaped = X_train.to_numpy().reshape(-1, 28, 28, 1)
X_train_normalized = X_train_reshaped / 255.0

# Convert labels to one-hot encoding
y_train_one_hot = tf.keras.utils.to_categorical(y_train, num_classes=10)

In [36]:
print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_one_hot:", y_train_one_hot.shape)


Shape of X_train_normalized: (42000, 28, 28, 1)
Shape of y_train_one_hot: (42000, 10)


In [ ]:
model = models.Sequential()

# Convolutional layers
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten and fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

In [ ]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

In [35]:
model

In [ ]:
model.fit(X_train_normalized, y_train_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=VALIDATION_SIZE)